In [1]:
import sys, os
from pathlib import Path
from PIL import Image, ImageDraw

sys.path.append(os.path.abspath("../"))
from yolo import YOLO

Using TensorFlow backend.


In [2]:
path_input_dir        = Path('../data/input/')
path_output_dir       = Path('../data/output/')
path_detection_output = Path(path_output_dir, 'detection_output.txt')
path_output_dir.mkdir(parents=True, exist_ok=True)

input_images = path_input_dir.glob('*')
# relevant_classes = {'car', 'truck'}

params = {
    'model_path':   f'../{YOLO.get_defaults("model_path")}',
    'anchors_path': f'../{YOLO.get_defaults("anchors_path")}',
    'classes_path': f'../{YOLO.get_defaults("classes_path")}',
    'font': '../font/FiraMono-Medium.otf',
    'image':  True,
}
model = YOLO(**params)

../model_data/yolo.h5 model, anchors, and classes loaded.


## TODO: one image per bounding box

In [3]:
def __get_row_format_string(img_path, *boxes):
    return f'{img_path} {" ".join(boxes)}'

# left, top, right, bottom, class_id  
def __get_box_format_string(x_min, y_min, x_max, y_max, class_id):
    return f'{x_min},{y_min},{x_max},{y_max},{class_id}'

def get_row(img_path, coords, classes):
    boxes_string = [
        __get_box_format_string(
            coords[i][0], 
            coords[i][1], 
            coords[i][2], 
            coords[i][3], 
            classes[i]) 
        for i in range(len(coords))]
    return __get_row_format_string(img_path, *boxes_string)

In [4]:
# Row format: image_file_path box1 box2 ... boxN;
# Box format: x_min,y_min,x_max,y_max,class_id (no space).
# out_img, _boxes, _scores, classes, coords, _labels = model.detect_image
# print(img_path.absolute())

output_rows = []

for img_path in input_images:
    img = Image.open(img_path)
    output_path = Path(path_output_dir, img_path.name)
    out_img, _, _, classes, coords, _ = model.detect_image(img.copy())
    
    output_rows += [get_row(img_path, coords, classes)]
    out_img.save(str(output_path))
    print('')

with open(path_detection_output, 'w+') as output_file:
    for row in output_rows:
        output_file.write(row + '\n')

(416, 416, 3)
Found 1 boxes for img
car 1.00 (0, 82) (3976, 2086)
3.0865582459955476

(416, 416, 3)
Found 2 boxes for img
person 1.00 (469, 27) (1021, 570)
person 1.00 (0, 92) (465, 576)
1.0754819050052902

